In [5]:
from sentence_transformers import SentenceTransformer, util

### Load Dataset

In [6]:
df = pd.read_csv('misinformation_papers.csv')

In [7]:
df.head(3)

,original_title,abstract,year,citations,id,is_EN
0,When Corrections Fail: The Persistence of Poli...,An extensive literature addresses citizen igno...,2010,901,2132553681,1
1,A postmodern Pandora's box: anti-vaccination m...,The Internet plays a large role in disseminati...,2010,440,2117485795,1
2,Spread of (Mis)Information in Social Networks,We provide a model to investigate the tension ...,2010,278,2120015072,1


### Load the Transformer Model

In [8]:
#load the distilbert model 
distilbert = SentenceTransformer('distilbert-base-uncased')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.54k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Some weights of the model checkpoint at C:\Users\jesse/.cache\torch\sentence_transformers\distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
#Compute embeddings for all paper's abstracts
corpus_embeddings = distilbert.encode(df['abstract'].to_list(), convert_to_tensor=True)

In [23]:
query = 'covid 19 and milk'

query_embedding = distilbert.encode(query)

search_hits = util.semantic_search(query_embedding, corpus_embeddings)
search_hits 

[[{'corpus_id': 6768, 'score': 0.6841555833816528},
  {'corpus_id': 2917, 'score': 0.6811084747314453},
  {'corpus_id': 8428, 'score': 0.6797125935554504},
  {'corpus_id': 5300, 'score': 0.6789922714233398},
  {'corpus_id': 7149, 'score': 0.6732285618782043},
  {'corpus_id': 1973, 'score': 0.6650269031524658},
  {'corpus_id': 6434, 'score': 0.663871169090271},
  {'corpus_id': 2326, 'score': 0.663203775882721},
  {'corpus_id': 2325, 'score': 0.663203775882721},
  {'corpus_id': 2329, 'score': 0.663203775882721}]]

In [49]:
#We define a function, given title & abstract, searches our corpus for relevant (similar) papers
def search_papers(query):
    
    query_embedding = distilbert.encode(query)

    search_hits = util.semantic_search(query_embedding, corpus_embeddings)
    search_hits = search_hits[0]  #Get the hits for the first query

    
    print("Most similar papers:")
    for hit in search_hits:
        related_paper = df.loc[hit['corpus_id']]
        print('Score: ',hit['score'], 
              'Title: ',related_paper['original_title'],
              'Abstract: ',related_paper['abstract'], 
              'Year: ',related_paper['year'])
        
        print('/n')

In [51]:
search_papers('Can covid be cured with alcohol')

Most similar papers:
Score:  0.7449579834938049 Title:  Finasteride: Facts and Myths Abstract:  Media reports, Internet sites and misinformation by alternative medicine practitioners recently contributed to a negative image associated with finasteride, leading to apprehension and concern among patients taking the medication. Frequently, even dermatologists seem to be hesitant to prescribe the drug for long periods, mistakenly fearing the occurrence of long-term side effects. Finasteride, either alone or in combination with topical minoxidil, is an excellent option for men with androgenetic alopecia, reducing hair loss, and/or restoring hair growth in 9 of 10 patients. The drug can be taken at any time of the day, with or without food. There are no reports of significant drug interactions or allergies. Because it is metabolized in the liver, the drug should be used with caution in patients with liver diseases, but there is no need to indicate liver function tests frequently yet. Year:  